在本章中，我们将学到如下内容：
+ 介绍常用分类算法的概念
+ 使用scikit-learn机器学习库
+ 选择机器学习算法时需要注意的问题  

5  
# 3.1 分类算法的选择
再次借用一下“没有免费午餐”理论：没有任何一种分类器可以在所有可能的应用场景下都有良好的表现  
总而言之，分类器的性能、计算能力和预测能力，在很大程度上都依赖于用于模型训练的相关数据。训练机器学习算法所涉及的五个主要步骤可以概述如下：  
1. 特征的选择  
2. 确定性能评价标准
3. 选择分类器及其优化算法
4. 对模型性能的评估
5. 算法的调优
5  

# 3.2 初涉scikit-learn的使用
## 使用sckit-learn训练感知器

In [ ]:
'''
python	sklearn	datasets	load_iris() feature_names
python	sklearn	datasets	load_iris() target
python	sklearn	datasets	load_iris() data
'''
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
iris.feature_names

为了评估训练得到的模型在未知数据上的表现，我们进一步将数据集划分为训练数据集和测试数据集 

In [ ]:
'''
python	sklearn	model_selection	train_test_split()
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 0)

在此，我们将使用scikit-learn的preprocessing模块中的standardscaler类对特征进行标准化处理  

In [ ]:
'''
5
'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

需要注意的是，我们要使用相同的缩放参数分别处理训练和测试数据集，以保证它们的值是彼此相当的  
在对训练数据做了标准化处理后，我们现在可以训练感知器模型了。scikit-learn中的大多数算法本身就使用一对多方法来支持多类别分类，因此，我们可以将三种鸢尾花的数据同时输入到感知器中    

In [ ]:
'''
python	sklearn	Linear Models	Perceptron() max_iter
python	sklearn	Linear Models	Perceptron() eta0
'''
from sklearn.linear_model import Perceptron
ppn = Perceptron(max_iter = 40, eta0 = 0.1, random_state = 0)
ppn.fit(X_train_std, y_train)

此模型的参数eta0与我们自行实现的感知器中的学习速率eta等价，而参数max_iter定义了迭代的次数  
与第2章中实现的感知器一样，使用scikit-learn完成模型的训练后，就可以在测试数据集上使用predict方法进行预测了 5 

In [ ]:
'''
python	str	format()	format() d
'''
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: {:d}'.format((y_test != y_pred).sum()))

许多机器学习从业者通常使用准确率而不是误分类来评判一个模型，它们关系如下：  
1 - 误分类率 = 准确率  
可以通过metrics模块计算感知器在测试数据集上的分类准确率：  

In [ ]:
'''
python	sklearn	Classification metrics	accuracy_score()
'''
from sklearn.metrics import accuracy_score
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

绘制刚刚训练过的模型的决策区域，我们做少许修改：使用小圆圈来高亮显示来自测试数据集的样本 5  

In [ ]:
'''
python	numpy	Array manipulation routines	unique()
python	matplotlib	colors	ListedColormap()
python	numpy	The N-dimensional array (ndarray)	min()
python	numpy	The N-dimensional array (ndarray)	max()
python	numpy	Array creation routines	meshgrid()
python	matplotlib	Pyplot function overview	contourf()
python	matplotlib	Pyplot function overview	scatter() alpha
python	matplotlib	collections	label
'''

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier,
                          test_idx = None, resolution = 0.02):
    # 设置散点生成器和色图
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # 绘制决策边界
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                          np.arange(x2_min, x2_max, resolution)) 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha = 0.4, cmap = cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # 绘制样本分类
    X_test, y_test = X[test_idx, :], y[test_idx]
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
                   alpha = 0.8, color = cmap(idx),
                   marker = markers[idx], label = cl)

    # 高亮测试样本
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], c = 'yellow',
                   alpha = 0.5, linewidth = 1, marker = 'o',
                   s = 55, label = 'test set')

In [ ]:
'''
python	numpy	Array manipulation routines	vstack()
python	numpy	Array manipulation routines	hstack()
python	matplotlib	Pyplot function overview	legend() loc

'''
import numpy as np
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X = X_combined_std,
                     y = y_combined,
                     classifier = ppn,
                     test_idx = range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

从结果呈现的图像中我们可以看到，无法通过一个线性决策边界完美区分三类样本  
对于无法完美线性可分的数据集，感知器算法将永远无法收敛，这也是在实践中一般不推荐使用感知器算法的原因 5
# 3.3 逻辑斯蒂回归中的类别概率
感知器在样本不是完全线性可分的情况下不会收敛的原因：在每次迭代过程中，总是存在至少一个分类错误的样本，从而导致额权重持续更新  
为了提高分类的效率，我们学习另外一种针对线性二类别分类问题的简单但更高效的算法：逻辑斯蒂回归 5  
## 3.3.1 初识逻辑斯蒂回归与条件概率
与感知器及Adaline类似，逻辑斯蒂回归模型也是适用于二类别分类问题的线性模型，通过一对多技术可以扩展到多类别分类  
我们先介绍一下几率比，它指的是特定事件发生的几率。用数学公式表示为$\frac{p}{(1-p)}$，其中p为正事件发生的概率  
正事件是指我们所要预测的事件，以一个患者患有某种疾病的概率为例，我们可以将正事件的类标标记为y=1。更进一步，我们可以定义logit函数，它是几率比的对数函数(log-odds, 对数几率)  
$$logit(p)=log\frac{p}{(1-p)}$$
logit函数的输入值范围介于区间[0,1]，它能将输入转换到整个实数范围内，由此可以将度数几率记为输入特征值的线性表达式 5  
$$logit(p(y=1|x))=w_0x_0+w_1x_1+\dots+w_mx_m=\sum_{i=0}^n{w_mx_m}=w^Tx$$
此处，$p(y=1|x)$是在给定特征x的条件下，某一个样本属于类别1的条件概率  
我们在此的真正目的是预测某一样本属于特定类别的概率，它是logit函数的反函数，也称作logistic函数，由于它的图像呈S形，因此有时也简称sigmoid函数  
$$\phi(z)=\frac{1}{1+e^{-z}}$$
这里的z为净输入，也就是样本特征与权重的线性组合，其计算方式为： 5  
$$z=w^Tx=w_0x_0+w_1x_1+\dots+w_mx_m$$
我们来绘制一下自变量取值介于区间[-7,7]的sigmoid函数的图像  

In [ ]:
'''
python	numpy	Mathematical functions	exp()
python	matplotlib	Pyplot function overview	axvline()
python	matplotlib	lines	color
python	matplotlib	Pyplot function overview	axhspan()
python	matplotlib	patches	Patch() facecolor
python	matplotlib	patches	Patch() alpha
python	matplotlib	patches	Patch() linestyle
python	matplotlib	Pyplot function overview	axhline()
python	matplotlib	lines	linestyle

'''
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
z = np.arange(-7, 7, 0.1)
phi_z = sigmoid(z)
plt.plot(z, phi_z)
plt.axvline(0.0, color='k')
plt.axhspan(0.0, 1.0, facecolor='1.0', alpha=1.0, linestyle='dotted')
plt.axhline(y=0.5, linestyle='dotted', color='black')
plt.axhline(y=0, linestyle='dotted', color='black')
plt.axhline(y=1.0, linestyle='dotted', color='black')
plt.yticks([0.0, 0.5, 1.0])
plt.ylim(-0.1, 1.1)
plt.xlabel('z')
plt.ylabel('$\phi (z)$')
plt.show()

执行上述代码，将会呈现一个S形曲线  
可以看到，当z趋向于无穷大时($z\to\infty$)时，$\varphi(z)$趋近于1，这是由于$e^{-z}$在z值极大的情况下其值变得极小。类似的，反之亦然 5  
由此，可以得出结论：sigmoid函数以实数值作为输入并将其映射到[0,1]区间，其拐点位于$\varphi(z)=0.5$处  
在Adaline中，我们使用恒等函数$\varphi(z)=z$作为激励函数。而在逻辑斯蒂回归中，只是简单地将前面提及的sigmoid函数作为激励函数，如下图所示：  
![3-1](../syn_pic/py_machine_learning/3-1.png)
在给定特征x及其权重w的情况下，sigmoid函数的输出给出了特定样本x属于类别1的概率$\varphi(z)=P(y=1|x;w)$  
预测得到的概率可以通过一个量化器(单位阶跃函数)简单地转换为二元输出： 5  
$$\hat{y}=\begin{cases}1&若\phi(z)\ge0.5 \\ 0&其他\end{cases}$$
对照前面的sigmoid函数的图像，它其实相当于：
$$\hat{y}=\begin{cases}1&若z\ge0.0 \\ 0&其他\end{cases}$$
对于许多应用实践来说，我们不但对类标预测感兴趣，而且对事件属于某一类别的概率进行预测也非常有用 5  
## 3.3.2 通过逻辑斯蒂回归模型的代价函数获得权重  
在上一章中，我们将其代价函数定义为误差平方和(SSE)  
$$J(w)=\sum_i{\frac{1}{2}(\phi(z^{(i)}-y^{(i)}))^2}$$
通过最小化此代价函数，我们可以得到Adaline分类模型的权重w。为了推导出逻辑斯蒂回归模型的代价函数，我们需要先定义一个最大似然函数L，假定数据集中的每个样本都是相互独立的，其计算公式如下：    
$$L(w)=P(y|x;w)=\prod_{i=1}^n{P(y^{(i)}|x^{(i)};w)}=(\phi(z^{(i)}))^{y^{(i)}}(1-\phi(z^{(i)}))^{1-y^{(i)}} $$
在实际应用中，很容易对此方程的自然对数进行最大化处理，故定义了对数似然函数： 5  